In [2]:
from __future__ import print_function
import assets.utils as u

from PIL import Image
import glob
from scipy.io import savemat, loadmat
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import normalized_mutual_info_score, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import time
from matplotlib import pyplot
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
labels_all = u.load_labels('C:/Users/ustundag/Github/2D-3D-Semantics/assets/semantic_labels.json')

In [4]:
# Function to map int labels to object classes
def assign_label(object):
    switcher = {
        'board':    0,
        'bookcase': 1,
        'chair':    2,
        'door':     3,
        'sofa':     4,
        'table':    5,
        'window':   6,
        '<UNK>':    7,
        'wall':     8,
        'beam':     9,
        'ceiling':  10,
        'clutter':  11,
        'column':   12,
        'floor':    13
    }

    return switcher.get(object, 14)

In [5]:
def convert_pixel_to_label(pixel):
    limit = int(len(labels_all))
    i = u.get_index(pixel)
    if i < limit:
        instance_label = labels_all[i]
        instance_label_as_dict = u.parse_label(instance_label)
        label = assign_label(instance_label_as_dict["instance_class"])
        return int(label)
    
    return 14 # default as missing data in case index is out of bounds in "labels.json" file

In [6]:
def save_labels(pixels):
    labels_gt = []
    for p in pixels:
        labels_gt.append(convert_pixel_to_label(p))

    return np.array(labels_gt)

In [7]:
# Truncate unnecessary objects in both datasets, like <UNK>, beam, ceiling, clutter, column, floor, wall.
# unnecessary objects after label=6, see assign_label() function.
def truncate_objects(pixels, labels):
    indices = np.argwhere(labels > 6)
    labels  = np.delete(labels, indices, axis=0)
    pixels  = np.delete(pixels, indices, axis=0)
    return pixels, labels
    '''
    a = np.array([[1, 2, 3],
               [4, 5, 6],
               [7, 8, 9]])
    b = np.array([8,4,9])
    indices = np.argwhere(b > 4)
    a = np.delete(a, indices, axis=0)
    b = np.delete(b, indices, axis=0)
    '''

### Crop individual objects using semantics as a mask

In [8]:
scale = 4  # The more scale down, the less number of unique pixels!
path = "C:\\Users\\ustundag\\GitHub\\2D-3D-Semantics\\area_3\\data\\semantic\\camera*.png"
i = 0
for sem in glob.glob(path):
    sem = "C:\\Users\\ustundag\\Desktop\\camera_81a9927c6b224f4bb5e6b9fbdcfae9c0_office_3_frame_25_domain_semantic.png"
    clear_output(wait=True)
    # Read and scale down images
    mask = np.array(Image.open(sem))
    mask = mask[::scale, ::scale]
    mask = mask.reshape(mask.shape[0]*mask.shape[1], mask.shape[2])
    rgb  = sem.replace("semantic", "rgb", 2) # find RGB image counterpart of semantic image
    img  = np.array(Image.open(rgb))
    img  = img[::scale, ::scale]
    img  = img.reshape(img.shape[0]*img.shape[1], img.shape[2])

    # Find unique pixels which means different objects and truncate unnecessary parts
    pixels = np.unique(mask, axis=0)
    labels = save_labels(pixels)
    pixels, labels = truncate_objects(pixels, labels)
    
    j = 0
    for p in pixels:
        segment = (mask == p).all(axis=1)
        obj = img * np.array([segment,segment,segment]).T
        lbl = convert_pixel_to_label(p)
        filename = 'image'+'_'+str(i)+'_'+'segment'+'_'+str(j)+'_'+'label'+'_'+str(lbl)
        Image.fromarray(obj.reshape(270,270,3)).save('C:/Users/ustundag/Github/2D-3D-Semantics/Individual_Objects/'+filename+'.png')
        j += 1
    
    i += 1 # holds image index
    display(str(i) + " / 3704")
    break

'1 / 3704'

In [9]:
# Masking example
"""
a = np.arange(12,24).reshape(4,3)
print(a)
b = np.arange(12).reshape(4,3)
b[1][0] = 6
print(b)
b = (b == [6,7,8]).all(axis=1)
#b = b.astype(np.int)
print(a.shape)
print(np.transpose(b))
a*np.array([b,b,b]).T
"""

'\na = np.arange(12,24).reshape(4,3)\nprint(a)\nb = np.arange(12).reshape(4,3)\nb[1][0] = 6\nprint(b)\nb = (b == [6,7,8]).all(axis=1)\n#b = b.astype(np.int)\nprint(a.shape)\nprint(np.transpose(b))\na*np.array([b,b,b]).T\n'